In [230]:
# ! pip install datasets evaluate transformers[sentencepiece]
# ! pip install accelerate 

In [231]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

# model_checkpoint = "distilbert-base-uncased"
model_checkpoint = "google-bert/bert-base-multilingual-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at google-bert/bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [232]:
text = "This is a great [MASK]."

In [233]:
import torch

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits

mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> This is a great success.'
'>>> This is a great thing.'
'>>> This is a great feat.'
'>>> This is a great adventure.'
'>>> This is a great deal.'


In [235]:
with open("data/100sentences_hin.txt", 'r', encoding='utf-8') as f:
    sent = [line.strip() for line in f]

print(type(sent))
print(sent[:3])

<class 'list'>
['निदान आप शायद नहीं जानते होंगे कि आपको एट्रियल फ़िब्रिलेशन (एएफआईबी) है', 'किसी अन्य कारण से स्वास्थ्य जांच कराने पर इस स्थिति का पता चल सकता है', 'यह त्वरित और दर्द रहित परीक्षण हृदय की विद्युत गतिविधि को मापता है']


In [203]:
# from datasets import Dataset
# from datasets import DatasetDict
# from sklearn.model_selection import train_test_split

# train, val, _, _ = train_test_split(sent, sent, train_size=0.8, random_state=1)

# train = [{"text": sentence} for sentence in train]
# train = Dataset.from_list(train)

# val = [{"text": sentence} for sentence in val]
# val = Dataset.from_list(val)

# dataset = DatasetDict({
#     "train": train,
#     "validation": val
# })

# print(dataset)

In [236]:
from datasets import Dataset
from datasets import DatasetDict

train = train = [{"text": sentence} for sentence in sent]
train = Dataset.from_list(train)

dataset = DatasetDict({
    "train": train,
    # "validation": val
})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 100
    })
})


In [237]:
sample = dataset["train"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f">>> {row['text']}'")

>>> क्या आप जो दवा लिख ​​रहे हैं उसका कोई सामान्य विकल्प है?'
>>> रक्त को पतला करने वाली दवाओं में वारफारिन जांटोवेन, एपिक्साबैन एलिकिस, डाबीगेट्रान प्रदाक्सा, एडोक्साबैन सावेसा और रिवरोक्साबैन ज़ेरेल्टो शामिल हैं।'
>>> आपके साथ जाने वाला कोई व्यक्ति आपको दी गई जानकारी याद रखने में मदद कर सकता है'


In [238]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = dataset.map(
    tokenize_function, batched=True, remove_columns=["text"]
)
tokenized_datasets

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 100
    })
})

In [239]:
tokenizer.model_max_length

512

In [240]:
chunk_size = 128

In [241]:
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> sentence {idx} length: {len(sample)}'")

'>>> sentence 0 length: 37'
'>>> sentence 1 length: 21'
'>>> sentence 2 length: 28'


In [242]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated sentences length: {total_length}'")

'>>> Concatenated sentences length: 86'


In [243]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 86'


In [247]:
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }

    result["labels"] = result["input_ids"].copy()
    return result

In [248]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 24
    })
})

In [249]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'स जोडत ह, जो परीकषण परिणामो को परिट या परदरशित करता ह [SEP] [CLS] जब आप अपनी नियमित गतिविधिया करत ह तो इस एक या दो दिन क लिए पहना जाता ह [SEP] [CLS] अनियमित हदय ताल का पता चलन पर कछ उपकरण सवचालित रप स रिकॉरड करत ह [SEP] [CLS] यह उपकरण तीन साल तक लगातार दिल की धडकन को रिकॉरड करता ह [SEP] [CLS] डिवाइस दिखाता ह कि जब आप अपनी दनिक गतिवि'

In [250]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [251]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f">>> {tokenizer.decode(chunk)}'")

>>> [CLS] [MASK]िदान आप श [MASK]द नही [MASK]त [MASK]ग कि आपको एटरियल फ [MASK]बरिलशन [MASK] एएफ [MASK]ईबी ) [MASK] [SEP] [CLS] किसी अनय [MASK] स सवासथय जाच करान पर इस सथिति का पता चल सकता ह [SEP] [CLS] यह तवरित [MASK] दरद रहित परीकषण हदय की विदयत गतिविधि को मापता ह [SEP] [CLS] इलकटरोड नामक [MASK]िपचिप पच छाती और कभी - कभी [MASK]ाहो [MASK] परो पर लगाए [MASK] ह [SEP] [CLS] तार [MASK]礼टरালিত [MASK] कपयटर'
>>> स जोडत ह [MASK] जो परीकषण पर [MASK]णामो को [MASK]िट या परदरशित करता ह [SEP] [CLS] जब आप अपनी नियमित गतिविधि [MASK] 蛆 ह तो इस एक या [MASK] दिन क [MASK] पहन [MASK] जाता ह [SEP] [CLS] अनियमित 絆दय [MASK]ाल [MASK] [MASK] चलन पर कछ उपकरण सवचालित रप स रिकॉरड करत ह [SEP] [CLS] यह उपकरण तीन साल तक लगातार दिल की धडक [MASK] को [MASK]िकॉरड करता ह [SEP] [CLS] डिवाइस दिखाता ह कि जब [MASK] [MASK] [MASK] दनिक गतिवि'


In [217]:
import collections
import numpy as np
from transformers import default_data_collator

def whole_word_masking_data_collator(features, wwm_probability = 0.2):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

In [218]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f">>> {tokenizer.decode(chunk)}")

>>> [CLS] நோய [MASK] [MASK] [MASK] [MASK] உஙகளுககு ஏடரியல ஃபைபரிலேஷன afib இருபபது [MASK] [MASK] [MASK] [MASK] தெரியாது [SEP] [CLS] மறறொரு காரணததிறகாக [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] செயயபபடும போது இநத நிலை கணடறியபபடலாம [SEP] [CLS] இநத விரைவான மறறும வலியறற சோதனை இதயததின மின செயலபாடடை அளவிடுகிறது [SEP] [CLS] மினமுனைகள எனபபடும ஒடடும இணைபபுகள மாரபிலும சில [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK]
>>> காலகளிலும வைககபபடுகினறன [SEP] [CLS] கமபிகள மினமுனைகளை கணினியுடன இணைககினறன, [MASK] சோதனை முடிவுகளை அசசிடுகிறது அலலது காணபிககும [SEP] [CLS] உஙகள [MASK] [MASK] [MASK] [MASK] செயலபாடுகளைச [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] ஒனறு [MASK] இரணடு நாடகளுககு அணியபபடும [SEP] [CLS] ஒழுஙகறற இதயத துடிபபு [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] சில சாதனஙகள தானாகவே பதிவு [MASK] [MASK] [MASK] [MASK] [SEP] [CLS] இநத [MASK] [MASK] [MASK] மூனறு ஆணடுகள வரை [MASK] [MASK] [MASK] து


In [219]:
train_size = 0.9
test_size = 0.1

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 25
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 3
    })
})

In [220]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned",
    # overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
)

In [221]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [222]:
trainer.train()

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 2.0333046913146973, 'eval_runtime': 0.1539, 'eval_samples_per_second': 19.496, 'eval_steps_per_second': 6.499, 'epoch': 1.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 1.349650502204895, 'eval_runtime': 0.1331, 'eval_samples_per_second': 22.533, 'eval_steps_per_second': 7.511, 'epoch': 2.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 1.8491617441177368, 'eval_runtime': 0.148, 'eval_samples_per_second': 20.273, 'eval_steps_per_second': 6.758, 'epoch': 3.0}
{'train_runtime': 32.11, 'train_samples_per_second': 2.336, 'train_steps_per_second': 0.093, 'train_loss': 1.6208845774332683, 'epoch': 3.0}


TrainOutput(global_step=3, training_loss=1.6208845774332683, metrics={'train_runtime': 32.11, 'train_samples_per_second': 2.336, 'train_steps_per_second': 0.093, 'train_loss': 1.6208845774332683, 'epoch': 3.0})

In [223]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model=model, tokenizer=tokenizer
)

In [224]:
# Diagnosis You may not know you have atrial fibrillation AFib
test_sentences =["Diagnosis [MASK] may not know you have atrial fibrillation AFib",
                 "Diagnosis You [MASK] not know you have atrial fibrillation AFib",
                 "Diagnosis You may not [MASK] you have atrial fibrillation AFib",
                 "Diagnosis You may not know you [MASK] atrial fibrillation AFib",
                 "Diagnosis You may not know you have [MASK] fibrillation AFib",
                 "Diagnosis You may not know you have atrial [MASK] AFib"]

for text in test_sentences:
    preds = mask_filler(text)
    print("-----")
    # for pred in preds:
    #     print(f"> {pred['sequence']}")
    print(preds[0]['sequence'])


-----
diagnosis you may not know you have atrial fibrillation afib
-----
diagnosis you do not know you have atrial fibrillation afib
-----
diagnosis you may not believe you have atrial fibrillation afib
-----
diagnosis you may not know you are atrial fibrillation afib
-----
diagnosis you may not know you have a fibrillation afib
-----
diagnosis you may not know you have atrial. afib


In [172]:
text = "निदान आप शायद नहीं जानते होंगे कि आपको एट्रियल फ़िब्रिलेशन (एएफआईबी) है"
masked = "निदान आप शायद [MASK] जानते होंगे कि [MASK] एट्रियल फ़िब्रिलेशन (एएफआईबी) है."

preds = mask_filler(masked)


print(f"Original text: \t {text}\n")

for pred in preds:
    # print(f"\t >{pred['sequence']}")
    print(pred)


Original text: 	 निदान आप शायद नहीं जानते होंगे कि आपको एट्रियल फ़िब्रिलेशन (एएफआईबी) है

[{'score': 0.22944827377796173, 'token': 29876, 'token_str': '##ा', 'sequence': '[CLS] निदान आप शायदा जानत होग कि [MASK] एटरियल [UNK] ( [UNK] ) ह. [SEP]'}, {'score': 0.17568902671337128, 'token': 29877, 'token_str': '##ि', 'sequence': '[CLS] निदान आप शायदि जानत होग कि [MASK] एटरियल [UNK] ( [UNK] ) ह. [SEP]'}, {'score': 0.17328883707523346, 'token': 29878, 'token_str': '##ी', 'sequence': '[CLS] निदान आप शायदी जानत होग कि [MASK] एटरियल [UNK] ( [UNK] ) ह. [SEP]'}, {'score': 0.11199919134378433, 'token': 29869, 'token_str': '##र', 'sequence': '[CLS] निदान आप शायदर जानत होग कि [MASK] एटरियल [UNK] ( [UNK] ) ह. [SEP]'}, {'score': 0.08817306905984879, 'token': 29863, 'token_str': '##न', 'sequence': '[CLS] निदान आप शायदन जानत होग कि [MASK] एटरियल [UNK] ( [UNK] ) ह. [SEP]'}]
[{'score': 0.21659012138843536, 'token': 29859, 'token_str': '##त', 'sequence': '[CLS] निदान आप शायद [MASK] जानत होग कित एटरियल [UNK] 

In [173]:
text = "Der Zustand kann festgestellt werden, wenn aus einem anderen Grund eine Gesundheitsuntersuchung durchgeführt wird"
masked = "Der Zustand [MASK] festgestellt werden, wenn aus einem anderen Grund eine Gesundheitsuntersuchung durchgeführt wird."

preds = mask_filler(masked)


print(f"Original text: \t {text}\n")

for pred in preds:
    # print(f"\t >{pred['sequence']}")
    print(pred)


Original text: 	 Der Zustand kann festgestellt werden, wenn aus einem anderen Grund eine Gesundheitsuntersuchung durchgeführt wird

{'score': 0.2608787417411804, 'token': 2063, 'token_str': '##e', 'sequence': 'der zustande festgestellt werden, wenn aus einem anderen grund eine gesundheitsuntersuchung durchgefuhrt wird.'}
{'score': 0.14552222192287445, 'token': 27412, 'token_str': '##liche', 'sequence': 'der zustandliche festgestellt werden, wenn aus einem anderen grund eine gesundheitsuntersuchung durchgefuhrt wird.'}
{'score': 0.058971669524908066, 'token': 2368, 'token_str': '##en', 'sequence': 'der zustanden festgestellt werden, wenn aus einem anderen grund eine gesundheitsuntersuchung durchgefuhrt wird.'}
{'score': 0.05674171447753906, 'token': 27665, 'token_str': 'eine', 'sequence': 'der zustand eine festgestellt werden, wenn aus einem anderen grund eine gesundheitsuntersuchung durchgefuhrt wird.'}
{'score': 0.044515468180179596, 'token': 4315, 'token_str': 'der', 'sequence': 'der

In [75]:
# ara
text = "يمكن اكتشاف الحالة عند إجراء فحص صحي لسبب آخر"
masked = "يمكن اكتشاف الحالة عند إجراء [MASK] صحي لسبب آخر"

preds = mask_filler(masked)


print(f"Original text: \t {text}\n")

for pred in preds:
    # print(f"\t >{pred['sequence']}")
    print(pred)


Original text: 	 يمكن اكتشاف الحالة عند إجراء فحص صحي لسبب آخر

{'score': 0.21959923207759857, 'token': 1268, 'token_str': '،', 'sequence': 'يمكن اكتشاف الحالة عند اجراء ، صحي لسبب اخر'}
{'score': 0.1942230463027954, 'token': 1270, 'token_str': 'ا', 'sequence': 'يمكن اكتشاف الحالة عند اجراء ا صحي لسبب اخر'}
{'score': 0.1261647790670395, 'token': 1298, 'token_str': 'و', 'sequence': 'يمكن اكتشاف الحالة عند اجراء و صحي لسبب اخر'}
{'score': 0.0734768807888031, 'token': 19433, 'token_str': '##ة', 'sequence': 'يمكن اكتشاف الحالة عند اجراءة صحي لسبب اخر'}
{'score': 0.03801721706986427, 'token': 1294, 'token_str': 'ل', 'sequence': 'يمكن اكتشاف الحالة عند اجراء ل صحي لسبب اخر'}


In [99]:
# fre
text = "À quelle fréquence dois-je subir un test de dépistage d'une maladie cardiaque ou de complications liées à la fibrillation auriculaire"
masked = "À quelle fréquence dois-je subir un test de dépistage d'une [MASK] cardiaque ou de complications liées à la fibrillation auriculaire"

preds = mask_filler(masked)


print(f"Original text: \t {text}\n")

for pred in preds:
    # print(f"\t >{pred['sequence']}")
    print(pred)


Original text: 	 À quelle fréquence dois-je subir un test de dépistage d'une maladie cardiaque ou de complications liées à la fibrillation auriculaire

{'score': 0.05648840218782425, 'token': 25207, 'token_str': 'nouvelle', 'sequence': "a quelle frequence dois - je subir un test de depistage d'une nouvelle cardiaque ou de complications liees a la fibrillation auriculaire"}
{'score': 0.036004677414894104, 'token': 2139, 'token_str': 'de', 'sequence': "a quelle frequence dois - je subir un test de depistage d'une de cardiaque ou de complications liees a la fibrillation auriculaire"}
{'score': 0.02528570033609867, 'token': 2474, 'token_str': 'la', 'sequence': "a quelle frequence dois - je subir un test de depistage d'une la cardiaque ou de complications liees a la fibrillation auriculaire"}
{'score': 0.01775709167122841, 'token': 3393, 'token_str': 'le', 'sequence': "a quelle frequence dois - je subir un test de depistage d'une le cardiaque ou de complications liees a la fibrillation auri

In [124]:
# jap
text = "心臓病または心房細動の合併症の検査はどのくらいの頻度で受けるべきですか"
masked = "心臓病または心房細動の合併症の検査はどのく[MASK]いの頻度で受けるべきですか"

preds = mask_filler(masked)


print(f"Original text: \t {text}\n")

for pred in preds:
    # print(f"\t >{pred['sequence']}")
    print(pred)

Original text: 	 心臓病または心房細動の合併症の検査はどのくらいの頻度で受けるべきですか

{'score': 0.2776617109775543, 'token': 30211, 'token_str': '##ら', 'sequence': '心 または 心 の 合 の はとのくら いの て けるへきてすか'}
{'score': 0.13419713079929352, 'token': 30213, 'token_str': '##る', 'sequence': '心 または 心 の 合 の はとのくる いの て けるへきてすか'}
{'score': 0.08000142872333527, 'token': 30191, 'token_str': '##て', 'sequence': '心 または 心 の 合 の はとのくて いの て けるへきてすか'}
{'score': 0.0701446533203125, 'token': 30217, 'token_str': '##ん', 'sequence': '心 または 心 の 合 の はとのくん いの て けるへきてすか'}
{'score': 0.041595570743083954, 'token': 30212, 'token_str': '##り', 'sequence': '心 または 心 の 合 の はとのくり いの て けるへきてすか'}


In [148]:
# man
text =   "健康的生活方式是预防心脏问题和心脏相关并发症的主要保护措施"
masked = "健康的生活方式是预防[MASK]脏问题和心脏相关并发症的主要保护措施"

preds = mask_filler(masked)


print(f"Original text: \t {text}\n")

for pred in preds:
    # print(f"\t >{pred['sequence']}")
    print(pred)

Original text: 	 健康的生活方式是预防心脏问题和心脏相关并发症的主要保护措施

{'score': 0.11782781779766083, 'token': 1849, 'token_str': '心', 'sequence': '健 的 生 方 心 和 心 相 的 主 保'}
{'score': 0.11191239953041077, 'token': 1916, 'token_str': '的', 'sequence': '健 的 生 方 的 和 心 相 的 主 保'}
{'score': 0.06738013029098511, 'token': 1740, 'token_str': '一', 'sequence': '健 的 生 方 一 和 心 相 的 主 保'}
{'score': 0.05129873752593994, 'token': 1796, 'token_str': '和', 'sequence': '健 的 生 方 和 和 心 相 的 主 保'}
{'score': 0.02993958815932274, 'token': 1855, 'token_str': '我', 'sequence': '健 的 生 方 我 和 心 相 的 主 保'}


In [174]:
# pol
text =   "Czy zawsze masz objawy, czy pojawiają się i znikają?"
masked = "Czy zawsze masz objawy, czy [MASK] się i znikają?"

preds = mask_filler(masked)


print(f"Original text: \t {text}\n")

for pred in preds:
    # print(f"\t >{pred['sequence']}")
    print(pred)

Original text: 	 Czy zawsze masz objawy, czy pojawiają się i znikają?

{'score': 0.04858575388789177, 'token': 23564, 'token_str': 'za', 'sequence': 'czy zawsze masz objawy, czy za sie i znikaja?'}
{'score': 0.046745702624320984, 'token': 2213, 'token_str': '##m', 'sequence': 'czy zawsze masz objawy, czym sie i znikaja?'}
{'score': 0.038410332053899765, 'token': 1062, 'token_str': 'z', 'sequence': 'czy zawsze masz objawy, czy z sie i znikaja?'}
{'score': 0.029569320380687714, 'token': 1010, 'token_str': ',', 'sequence': 'czy zawsze masz objawy, czy, sie i znikaja?'}
{'score': 0.027179347351193428, 'token': 2480, 'token_str': '##z', 'sequence': 'czy zawsze masz objawy, czyz sie i znikaja?'}


In [225]:
# tam
text =   "நான் பின்பற்ற வேண்டிய வேறு ஏதேனும் கட்டுப்பாடுகள் உள்ளனவா"
masked = "நான் பின்பற்ற வேண்டிய [MASK] ஏதேனும் கட்டுப்பாடுகள் உள்ளனவா"

preds = mask_filler(masked)


print(f"Original text: \t {text}\n")

for pred in preds:
    # print(f"\t >{pred['sequence']}")
    print(pred)

Original text: 	 நான் பின்பற்ற வேண்டிய வேறு ஏதேனும் கட்டுப்பாடுகள் உள்ளனவா

{'score': 0.15485689043998718, 'token': 39216, 'token_str': '##தாக', 'sequence': 'நான பினபறற வேணடியதாக ஏதேனும கடடுபபாடுகள உளளனவா'}
{'score': 0.06965184956789017, 'token': 16112, 'token_str': '##ால', 'sequence': 'நான பினபறற வேணடியால ஏதேனும கடடுபபாடுகள உளளனவா'}
{'score': 0.05663551762700081, 'token': 79264, 'token_str': 'அனைதது', 'sequence': 'நான பினபறற வேணடிய அனைதது ஏதேனும கடடுபபாடுகள உளளனவா'}
{'score': 0.042365241795778275, 'token': 14814, 'token_str': '##து', 'sequence': 'நான பினபறற வேணடியது ஏதேனும கடடுபபாடுகள உளளனவா'}
{'score': 0.03979815915226936, 'token': 49683, 'token_str': '##வரகள', 'sequence': 'நான பினபறற வேணடியவரகள ஏதேனும கடடுபபாடுகள உளளனவா'}
